In [1]:
# ======================================================
# 1. Mount Google Drive
# ======================================================
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# ======================================================
# 2. Import libraries
# ======================================================
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
# ======================================================
# 3. Dataset path (ubah sesuai lokasi kamu)
# ======================================================
path_org = "/content/drive/MyDrive/CapstoneProject/signatures/full_org"
path_forg = "/content/drive/MyDrive/CapstoneProject/signatures/full_forg"

In [14]:
# ======================================================
# 4. Fungsi bantu untuk load & augment gambar
# ======================================================
import tensorflow as tf

def load_image(path, size=(155, 220)):
    img = load_img(path, color_mode='grayscale', target_size=size)
    img = img_to_array(img) / 255.0
    # augmentasi ringan biar model lebih general
    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_contrast(img, 0.9, 1.1)
    return img

def get_id(filename):
    # contoh nama: original_8_2.png -> ID = 8
    parts = filename.split('_')
    return int(parts[1])


In [15]:
# ======================================================
# 5. Buat pasangan data
# ======================================================
org_files = [f for f in os.listdir(path_org) if f.endswith('.png')]
forg_files = [f for f in os.listdir(path_forg) if f.endswith('.png')]

org_dict = {}
forg_dict = {}

for f in org_files:
    pid = get_id(f)
    org_dict.setdefault(pid, []).append(f)

for f in forg_files:
    pid = get_id(f)
    forg_dict.setdefault(pid, []).append(f)

pairs, labels = [], []

# Positif (asli vs asli)
for pid, imgs in org_dict.items():
    for i in range(len(imgs) - 1):
        img1 = load_image(os.path.join(path_org, imgs[i]))
        img2 = load_image(os.path.join(path_org, imgs[i + 1]))
        pairs.append([img1, img2])
        labels.append(1)

# Negatif (asli vs palsu)
for pid, imgs in forg_dict.items():
    if pid in org_dict:
        for forg_img in imgs:
            img1 = load_image(os.path.join(path_org, random.choice(org_dict[pid])))
            img2 = load_image(os.path.join(path_forg, forg_img))
            pairs.append([img1, img2])
            labels.append(0)

pairs = np.array(pairs)
labels = np.array(labels)

print(f"✅ Total pairs: {len(pairs)} | Positive: {np.sum(labels)} | Negative: {len(labels) - np.sum(labels)}")

✅ Total pairs: 2585 | Positive: 1265 | Negative: 1320


In [16]:
# ======================================================
# 6. Split data
# ======================================================
X_train, X_test, y_train, y_test = train_test_split(pairs, labels, test_size=0.2, random_state=42)

train_img1 = np.array([x[0] for x in X_train])
train_img2 = np.array([x[1] for x in X_train])
test_img1 = np.array([x[0] for x in X_test])
test_img2 = np.array([x[1] for x in X_test])


In [17]:
# ======================================================
# 7. Bangun backbone CNN
# ======================================================
def build_base_network(input_shape=(155, 220, 1)):
    input = Input(shape=input_shape)
    x = Conv2D(32, (7,7), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(64, (5,5), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (3,3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(512, activation='sigmoid')(x)
    return Model(input, x)

base_network = build_base_network()




In [18]:
# ======================================================
# 8. Definisikan jarak Euclidean dan Contrastive Loss
# ======================================================
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)



In [19]:
# ======================================================
# 9. Bangun model Siamese
# ======================================================
input_a = Input(shape=(155, 220, 1))
input_b = Input(shape=(155, 220, 1))

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, name='euclidean_distance')([processed_a, processed_b])

siamese_net = Model([input_a, input_b], distance)
siamese_net.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(0.0001))

siamese_net.summary()



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 155, 220,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 155, 220,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 512)       │ 106,098,9… │ input_layer_4[0]… │
│ (Functional)        │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ euclidean_distance  │ (None, 1)         │          0 │ functional_2[0][… │
│ (Lambda)            │                   │            │ functional_2[1][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 106,098,944 (404.74 MB)

 Trainable params: 106,098,944 (404.74 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# ======================================================
# 10. Callback (checkpoint + early stopping)
# ======================================================
checkpoint_path = "/content/drive/MyDrive/CapstoneProject/signature_siamese_best.keras"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True)
]

In [21]:
# ======================================================
# 11. Training
# ======================================================
history = siamese_net.fit(
    [train_img1, train_img2],
    y_train,
    validation_data=([test_img1, test_img2], y_test),
    batch_size=16,
    epochs=25,
    callbacks=callbacks
)

Epoch 1/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 52s 349ms/step - loss: 0.2714 - val_loss: 0.1526
Epoch 2/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 41s 319ms/step - loss: 0.0943 - val_loss: 0.0544
Epoch 3/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 0.0362 - val_loss: 0.1159
Epoch 4/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 0.0317 - val_loss: 0.0760
Epoch 5/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 41s 315ms/step - loss: 0.0174 - val_loss: 0.0468
Epoch 6/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 48s 368ms/step - loss: 0.0211 - val_loss: 0.0433
Epoch 7/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 34s 262ms/step - loss: 0.0106 - val_loss: 0.0363
Epoch 8/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0074 - val_loss: 0.0378
Epoch 9/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0075 - val_loss: 0.0384
Epoch 10/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 32s 246ms/step - loss: 0.0045 - val_loss: 0.0349
Epoch 11/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 0.0085 - val_loss: 0.0431
Epoch 12/25
130/130 ━━━━━━━━━

In [23]:
# Evaluate manual accuracy
y_pred_train = siamese_net.predict([train_img1, train_img2])
y_pred_test = siamese_net.predict([test_img1, test_img2])

train_acc = compute_accuracy(y_train, y_pred_train)
test_acc = compute_accuracy(y_test, y_pred_test)

print(f"Training accuracy: {train_acc:.4f}")
print(f"Testing accuracy: {test_acc:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Training accuracy: 1.0000
Testing accuracy: 0.9884


In [22]:
# ======================================================
# 12. Simpan model final ke format .keras
# ======================================================
final_model_path = "/content/drive/MyDrive/CapstoneProject/signature_siamese_final.keras"
siamese_net.save(final_model_path)

print(f"\n✅ Training selesai. Model disimpan di:\n{final_model_path}")


✅ Training selesai. Model disimpan di:
/content/drive/MyDrive/CapstoneProject/signature_siamese_final.keras
